In [1]:
import csv
import pandas as pd
import numpy as np
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error
# Importamos utilidades y modelos de sklearn
from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold


# Se levanta el set de train generado

In [2]:
data_set = pd.read_csv("train_set_xgb.csv")
#X = data_set.loc[:, (data_set.columns != 'precio') & (data_set.columns != 'factor_pm')]
X = data_set.loc[:, (data_set.columns != 'precio')]
y = data_set.loc[:, ['precio']]

In [3]:
data_set.columns

Index(['antiguedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos',
       'metrostotales', 'idzona', 'lat', 'lng', 'gimnasio', 'usosmultiples',
       'piscina', 'escuelascercanas', 'centroscomercialescercanos', 'precio',
       'tipodepropiedad_1', 'tipodepropiedad_2', 'tipodepropiedad_3',
       'tipodepropiedad_4', 'tipodepropiedad_5', 'tipodepropiedad_6',
       'tipodepropiedad_7', 'tipodepropiedad_8', 'tipodepropiedad_9',
       'tipodepropiedad_10', 'tipodepropiedad_11', 'tipodepropiedad_12',
       'tipodepropiedad_13', 'tipodepropiedad_14', 'tipodepropiedad_15',
       'tipodepropiedad_16', 'tipodepropiedad_17', 'tipodepropiedad_18',
       'tipodepropiedad_19', 'tipodepropiedad_20', 'tipodepropiedad_21',
       'tipodepropiedad_22', 'tipodepropiedad_23', 'tipodepropiedad_24',
       'tipodepropiedad_25', 'provincia_1', 'provincia_2', 'provincia_3',
       'provincia_4', 'provincia_5', 'provincia_6', 'provincia_7',
       'provincia_8', 'provincia_9', 'provincia_10', 

In [4]:
X.columns

Index(['antiguedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos',
       'metrostotales', 'idzona', 'lat', 'lng', 'gimnasio', 'usosmultiples',
       'piscina', 'escuelascercanas', 'centroscomercialescercanos',
       'tipodepropiedad_1', 'tipodepropiedad_2', 'tipodepropiedad_3',
       'tipodepropiedad_4', 'tipodepropiedad_5', 'tipodepropiedad_6',
       'tipodepropiedad_7', 'tipodepropiedad_8', 'tipodepropiedad_9',
       'tipodepropiedad_10', 'tipodepropiedad_11', 'tipodepropiedad_12',
       'tipodepropiedad_13', 'tipodepropiedad_14', 'tipodepropiedad_15',
       'tipodepropiedad_16', 'tipodepropiedad_17', 'tipodepropiedad_18',
       'tipodepropiedad_19', 'tipodepropiedad_20', 'tipodepropiedad_21',
       'tipodepropiedad_22', 'tipodepropiedad_23', 'tipodepropiedad_24',
       'tipodepropiedad_25', 'provincia_1', 'provincia_2', 'provincia_3',
       'provincia_4', 'provincia_5', 'provincia_6', 'provincia_7',
       'provincia_8', 'provincia_9', 'provincia_10', 'provincia

In [5]:
y.columns

Index(['precio'], dtype='object')

# Se separa en train y en test

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1)

In [7]:
len(X_train)

168000

In [8]:
len(X_test)

72000

## Armamos el dataset sobre el cual cada XGBoost va a subsamplear con reposición.

### Esto se hace sobre el X_train con su respectivo label.

In [9]:
data_for_xgb = pd.concat([X_train, y_train], axis = 1)
data_for_xgb.head()

,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,lat,lng,gimnasio,...,anio_publ,mes_publ,cuenta,planta,cuarto,tiene,baño,departamento,excelente,precio
239101,0.0,3.0,2.0,3.0,196.0,160.0,NaN,NaN,NaN,0.0,...,2014,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1800000.0
96221,20.0,4.0,2.0,2.0,245.0,196.0,104734.0,NaN,NaN,0.0,...,2012,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1500000.0
144453,5.0,3.0,1.0,2.0,108.0,120.0,58952.0,NaN,NaN,0.0,...,2015,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,850000.0
15620,10.0,3.0,1.0,3.0,200.0,147.0,49066.0,NaN,NaN,0.0,...,2016,10,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1550000.0
73157,NaN,NaN,0.0,NaN,NaN,250.0,24892.0,19.23339,-99.166992,0.0,...,2016,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,700000.0


# Se entrenan n XGB

Se define la cantidad de xgb a entrenar

In [10]:
n = 100

Se utilizan los valores encontrados en el Grid Search

In [11]:
xgb_created = []

In [12]:
for i in range(n):
    d_for_current_xgb = data_for_xgb.sample(frac = 1, replace = True, random_state = i)
    X_for_current_xgb = d_for_current_xgb.loc[:, (d_for_current_xgb.columns != 'precio')]
    y_for_current_xgb = d_for_current_xgb.loc[:, ['precio']]
    current_xgb = xgb.XGBRegressor(objective ='reg:squarederror', 
                                colsample_bytree = 0.8, 
                                gamma = 0.5,
                                 learning_rate = 0.1,
                                 max_depth = 5,
                                 min_child_weight = 10,
                                 subsample = 1.0, n_estimators = 2400, n_jobs = -1, random_state = i)
    current_xgb.fit(X_for_current_xgb, y_for_current_xgb)
    current_xgb.save_model("xgb_{}".format(i))
    xgb_created.append(current_xgb)
    print("XGB number {} created".format(i))

XGB number 0 created
XGB number 1 created
XGB number 2 created
XGB number 3 created
XGB number 4 created
XGB number 5 created
XGB number 6 created
XGB number 7 created
XGB number 8 created
XGB number 9 created
XGB number 10 created
XGB number 11 created
XGB number 12 created
XGB number 13 created
XGB number 14 created
XGB number 15 created
XGB number 16 created
XGB number 17 created
XGB number 18 created
XGB number 19 created
XGB number 20 created
XGB number 21 created
XGB number 22 created
XGB number 23 created
XGB number 24 created
XGB number 25 created
XGB number 26 created
XGB number 27 created
XGB number 28 created
XGB number 29 created
XGB number 30 created
XGB number 31 created
XGB number 32 created
XGB number 33 created
XGB number 34 created
XGB number 35 created
XGB number 36 created
XGB number 37 created
XGB number 38 created
XGB number 39 created
XGB number 40 created
XGB number 41 created
XGB number 42 created
XGB number 43 created
XGB number 44 created
XGB number 45 create

#### Se usa un subsample de igual tamaño que X_train

### Disclaimer: No se testea el armado de cada XGBoost porque asumo que funciona debido a que es el mismo modelo con los mismos hiperparametros que se usó anteriormente y lo único que cambia es que se está entrenando sobre un subsample de los datos originales.

# Se arma el DF de precios para entrenar el RF

In [13]:
RF_train = pd.DataFrame(index = X_train.index.copy())

In [14]:
for i in range(n):
    current_price_pred = xgb_created[i].predict(X_train)
    RF_train['pred_{}'.format(i)] = current_price_pred
    print("xgb_{} finished pred".format(i))
RF_train.head()

xgb_0 finished pred
xgb_1 finished pred
xgb_2 finished pred
xgb_3 finished pred
xgb_4 finished pred
xgb_5 finished pred
xgb_6 finished pred
xgb_7 finished pred
xgb_8 finished pred
xgb_9 finished pred
xgb_10 finished pred
xgb_11 finished pred
xgb_12 finished pred
xgb_13 finished pred
xgb_14 finished pred
xgb_15 finished pred
xgb_16 finished pred
xgb_17 finished pred
xgb_18 finished pred
xgb_19 finished pred
xgb_20 finished pred
xgb_21 finished pred
xgb_22 finished pred
xgb_23 finished pred
xgb_24 finished pred
xgb_25 finished pred
xgb_26 finished pred
xgb_27 finished pred
xgb_28 finished pred
xgb_29 finished pred
xgb_30 finished pred
xgb_31 finished pred
xgb_32 finished pred
xgb_33 finished pred
xgb_34 finished pred
xgb_35 finished pred
xgb_36 finished pred
xgb_37 finished pred
xgb_38 finished pred
xgb_39 finished pred
xgb_40 finished pred
xgb_41 finished pred
xgb_42 finished pred
xgb_43 finished pred
xgb_44 finished pred
xgb_45 finished pred
xgb_46 finished pred
xgb_47 finished pred
xg

,pred_0,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6,pred_7,pred_8,pred_9,...,pred_90,pred_91,pred_92,pred_93,pred_94,pred_95,pred_96,pred_97,pred_98,pred_99
239101,1.689986e+06,1766534.000,1.735717e+06,1715772.000,1.662306e+06,1.766623e+06,1.804056e+06,1617844.125,1.637010e+06,1738870.000,...,1821229.250,1.684417e+06,1.665304e+06,1676015.000,1838827.000,1794825.625,1827841.750,1842697.375,1.644829e+06,1.688090e+06
96221,1.749084e+06,1760283.125,1.743024e+06,1864231.875,1.783093e+06,1.743951e+06,1.847932e+06,1731897.250,1.732194e+06,1771472.000,...,1652129.750,1.862086e+06,1.778841e+06,1852972.625,1562068.375,1770447.375,1751814.625,1864356.000,1.671443e+06,1.761365e+06
144453,9.495233e+05,936526.250,8.750789e+05,848156.750,9.261204e+05,9.235599e+05,8.348517e+05,775759.750,9.075124e+05,968468.250,...,884079.875,8.463847e+05,8.154707e+05,886198.250,912867.750,974787.750,866842.625,920774.500,7.197323e+05,8.658220e+05
15620,1.907250e+06,2018018.375,2.129554e+06,2069715.000,1.899689e+06,2.037284e+06,1.928701e+06,1980490.875,1.874637e+06,2000020.375,...,1974532.250,1.885160e+06,1.860198e+06,1839548.125,2049893.250,2074825.875,2055814.000,2042658.875,2.016976e+06,1.990134e+06
73157,1.250160e+06,1706558.125,6.767887e+05,1090993.375,1.500844e+06,9.150998e+05,1.703472e+06,1115108.625,1.221259e+06,2409765.750,...,1905811.250,1.132271e+06,1.800622e+06,1839616.625,1771405.125,1746415.875,845541.500,897758.250,1.912185e+06,8.427619e+05


In [15]:
len(RF_train)

168000

In [16]:
len(y_train)

168000

## Exporto el RF_train

In [17]:
RF_train.to_csv("RF_train", index = False)

In [9]:
y_train.to_csv("y_train", index = False)

In [10]:
y_test.to_csv("y_test", index = False)

## Armo el DF para RF a partir de X_train

In [18]:
RF_test = pd.DataFrame(index = X_test.index.copy())


In [19]:
for i in range(n):
    current_price_pred = xgb_created[i].predict(X_test)
    RF_test['pred_{}'.format(i)] = current_price_pred
RF_test.head()

,pred_0,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6,pred_7,pred_8,pred_9,...,pred_90,pred_91,pred_92,pred_93,pred_94,pred_95,pred_96,pred_97,pred_98,pred_99
132827,1.659478e+06,1.471060e+06,1.455868e+06,1.862524e+06,1625050.75,1.505616e+06,1.762531e+06,1.662114e+06,1517567.250,1.840291e+06,...,1.619646e+06,1.637417e+06,1.688696e+06,1.692034e+06,1.576439e+06,1.560470e+06,1.457542e+06,1384460.250,2.058913e+06,1.528536e+06
210008,2.852040e+06,2.427407e+06,2.534238e+06,2.790467e+06,2607273.50,2.561150e+06,2.428484e+06,2.616929e+06,2462442.250,2.694016e+06,...,2.424282e+06,2.593619e+06,2.473534e+06,2.548829e+06,2.225817e+06,2.976241e+06,2.532902e+06,2539388.750,2.578002e+06,2.440599e+06
189056,2.181965e+06,2.135074e+06,1.959478e+06,2.361723e+06,2639389.00,2.744442e+06,2.361738e+06,2.178900e+06,2361966.000,2.730512e+06,...,2.203554e+06,1.979871e+06,1.857047e+06,2.089209e+06,2.828616e+06,2.180816e+06,2.402476e+06,1661171.500,2.629078e+06,2.707134e+06
131466,1.195996e+06,1.269788e+06,1.232923e+06,1.250264e+06,1192167.25,1.328625e+06,1.333840e+06,1.120925e+06,1624995.375,1.462925e+06,...,1.291122e+06,1.256644e+06,1.389990e+06,1.281326e+06,1.554499e+06,1.304531e+06,1.271230e+06,2119825.000,1.201555e+06,1.222957e+06
207302,1.181610e+05,3.444742e+05,2.312118e+05,3.113812e+05,268830.00,2.040469e+05,2.054723e+05,2.051386e+05,358404.500,1.923582e+05,...,4.239466e+05,2.483484e+05,2.063649e+05,2.584964e+05,4.133190e+05,2.326205e+05,3.403198e+05,376704.375,2.947268e+05,3.599904e+05


# Se exporta RF_test

In [20]:
RF_test.to_csv("RF_test", index = False)